In [ ]:
import findspark
findspark.init()

In [1]:
from pyspark.sql import SparkSession

# Build the SparkSession
spark = SparkSession.builder \
   .master("local") \
   .appName("dat gi cung duoc") \
   .config("spark.executor.memory", "1gb") \
   .getOrCreate()
   
sc = spark.sparkContext

In [2]:
rdd = sc.textFile('/Users/Storage/Vicohub/Data/Iris.csv')
rdd.take(5)

['sepal_length,sepal_width,petal_length,petal_width,species',
 '5.1,3.5,1.4,0.2,Iris-setosa',
 '4.9,3.0,1.4,0.2,Iris-setosa',
 '4.7,3.2,1.3,0.2,Iris-setosa',
 '4.6,3.1,1.5,0.2,Iris-setosa']

In [3]:
df = spark.read.csv('/Users/Storage/Vicohub/Data/Iris.csv', header=True)
df.show()

+------------+-----------+------------+-----------+-----------+
|sepal_length|sepal_width|petal_length|petal_width|    species|
+------------+-----------+------------+-----------+-----------+
|         5.1|        3.5|         1.4|        0.2|Iris-setosa|
|         4.9|        3.0|         1.4|        0.2|Iris-setosa|
|         4.7|        3.2|         1.3|        0.2|Iris-setosa|
|         4.6|        3.1|         1.5|        0.2|Iris-setosa|
|         5.0|        3.6|         1.4|        0.2|Iris-setosa|
|         5.4|        3.9|         1.7|        0.4|Iris-setosa|
|         4.6|        3.4|         1.4|        0.3|Iris-setosa|
|         5.0|        3.4|         1.5|        0.2|Iris-setosa|
|         4.4|        2.9|         1.4|        0.2|Iris-setosa|
|         4.9|        3.1|         1.5|        0.1|Iris-setosa|
|         5.4|        3.7|         1.5|        0.2|Iris-setosa|
|         4.8|        3.4|         1.6|        0.2|Iris-setosa|
|         4.8|        3.0|         1.4| 

In [4]:
def dosomething(x):
    return (DenseVector(x[:4]), x[4])

In [5]:
from pyspark.ml.linalg import DenseVector

# Define the `input_data` 
# input_data = df.rdd.map(lambda x: (DenseVector(x[:4]), x[4]))

input_data = df.rdd.map(dosomething)

# Replace `df` with the new DataFrame
df1 = spark.createDataFrame(input_data, ["X", "Y"])
df1.show(5)

+-----------------+-----------+
|                X|          Y|
+-----------------+-----------+
|[5.1,3.5,1.4,0.2]|Iris-setosa|
|[4.9,3.0,1.4,0.2]|Iris-setosa|
|[4.7,3.2,1.3,0.2]|Iris-setosa|
|[4.6,3.1,1.5,0.2]|Iris-setosa|
|[5.0,3.6,1.4,0.2]|Iris-setosa|
+-----------------+-----------+
only showing top 5 rows



In [7]:
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol="Y", outputCol="Yn")
indexed = indexer.fit(df1).transform(df1)
indexed.show(60)

+-----------------+---------------+---+
|                X|              Y| Yn|
+-----------------+---------------+---+
|[5.1,3.5,1.4,0.2]|    Iris-setosa|0.0|
|[4.9,3.0,1.4,0.2]|    Iris-setosa|0.0|
|[4.7,3.2,1.3,0.2]|    Iris-setosa|0.0|
|[4.6,3.1,1.5,0.2]|    Iris-setosa|0.0|
|[5.0,3.6,1.4,0.2]|    Iris-setosa|0.0|
|[5.4,3.9,1.7,0.4]|    Iris-setosa|0.0|
|[4.6,3.4,1.4,0.3]|    Iris-setosa|0.0|
|[5.0,3.4,1.5,0.2]|    Iris-setosa|0.0|
|[4.4,2.9,1.4,0.2]|    Iris-setosa|0.0|
|[4.9,3.1,1.5,0.1]|    Iris-setosa|0.0|
|[5.4,3.7,1.5,0.2]|    Iris-setosa|0.0|
|[4.8,3.4,1.6,0.2]|    Iris-setosa|0.0|
|[4.8,3.0,1.4,0.1]|    Iris-setosa|0.0|
|[4.3,3.0,1.1,0.1]|    Iris-setosa|0.0|
|[5.8,4.0,1.2,0.2]|    Iris-setosa|0.0|
|[5.7,4.4,1.5,0.4]|    Iris-setosa|0.0|
|[5.4,3.9,1.3,0.4]|    Iris-setosa|0.0|
|[5.1,3.5,1.4,0.3]|    Iris-setosa|0.0|
|[5.7,3.8,1.7,0.3]|    Iris-setosa|0.0|
|[5.1,3.8,1.5,0.3]|    Iris-setosa|0.0|
|[5.4,3.4,1.7,0.2]|    Iris-setosa|0.0|
|[5.1,3.7,1.5,0.4]|    Iris-setosa|0.0|


In [8]:
(trainingData, testData) = indexed.randomSplit([0.7, 0.3])

In [12]:
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(labelCol="Yn", 
                            featuresCol="X", 
                            numTrees=10)
model = rf.fit(trainingData)

In [13]:
predictions = model.transform(testData)

In [14]:
predictions.show(5)

+-----------------+-----------+---+--------------+-------------+----------+
|                X|          Y| Yn| rawPrediction|  probability|prediction|
+-----------------+-----------+---+--------------+-------------+----------+
|[4.3,3.0,1.1,0.1]|Iris-setosa|0.0|[10.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
|[4.4,2.9,1.4,0.2]|Iris-setosa|0.0|[10.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
|[4.4,3.0,1.3,0.2]|Iris-setosa|0.0|[10.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
|[4.6,3.1,1.5,0.2]|Iris-setosa|0.0|[10.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
|[4.6,3.4,1.4,0.3]|Iris-setosa|0.0|[10.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
+-----------------+-----------+---+--------------+-------------+----------+
only showing top 5 rows



In [15]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

predictions.select("Yn", "prediction")

evaluator = MulticlassClassificationEvaluator(
    labelCol="Yn", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Accuracy = %g" % (accuracy))

Test Accuracy = 0.883721


### Boston Housing

In [18]:
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import DoubleType, IntegerType, StringType

schema = StructType([
    StructField("CRIM", DoubleType()),
    StructField("ZN", DoubleType()),
    StructField("INDUS", DoubleType()),
    StructField("CHAS", DoubleType()),
    StructField("NOX", DoubleType()),
    StructField("RM", DoubleType()),
    StructField("AGE", DoubleType()),
    StructField("DIS", DoubleType()),
    StructField("RAD", DoubleType()),
    StructField("TAX", DoubleType()),
    StructField("PTRATIO", DoubleType()),
    StructField("B", DoubleType()),
    StructField("LSTAT", DoubleType()),
    StructField("MEDV", DoubleType())
])

df = spark.read.format("com.databricks.spark.csv").schema(schema).option("header", "true") \
    .load("/Users/Storage/Vicohub/Data/housing/boston_housing.csv")
# df = spark.read.csv('/Users/Storage/Vicohub/Data/housing/boston_housing.csv', header=True)
df.show(5)

+-------+----+-----+----+-----+-----+----+------+---+-----+-------+------+-----+----+
|   CRIM|  ZN|INDUS|CHAS|  NOX|   RM| AGE|   DIS|RAD|  TAX|PTRATIO|     B|LSTAT|MEDV|
+-------+----+-----+----+-----+-----+----+------+---+-----+-------+------+-----+----+
|0.00632|18.0| 2.31| 0.0|0.538|6.575|65.2|  4.09|1.0|296.0|   15.3| 396.9| 4.98|24.0|
|0.02731| 0.0| 7.07| 0.0|0.469|6.421|78.9|4.9671|2.0|242.0|   17.8| 396.9| 9.14|21.6|
|0.02729| 0.0| 7.07| 0.0|0.469|7.185|61.1|4.9671|2.0|242.0|   17.8|392.83| 4.03|34.7|
|0.03237| 0.0| 2.18| 0.0|0.458|6.998|45.8|6.0622|3.0|222.0|   18.7|394.63| 2.94|33.4|
|0.06905| 0.0| 2.18| 0.0|0.458|7.147|54.2|6.0622|3.0|222.0|   18.7| 396.9| 5.33|36.2|
+-------+----+-----+----+-----+-----+----+------+---+-----+-------+------+-----+----+
only showing top 5 rows



In [19]:
df.printSchema

<bound method DataFrame.printSchema of DataFrame[CRIM: double, ZN: double, INDUS: double, CHAS: double, NOX: double, RM: double, AGE: double, DIS: double, RAD: double, TAX: double, PTRATIO: double, B: double, LSTAT: double, MEDV: double]>

In [20]:
from pyspark.ml.linalg import DenseVector
data = df.rdd.map(lambda x: (DenseVector(x[:13]), x[13]))
df1 = spark.createDataFrame(data, ["X", "Y"])
(trainingData, testData) = df1.randomSplit([0.7, 0.3])

In [11]:
trainingData.show(5)

+--------------------+----+
|                   X|   Y|
+--------------------+----+
|[0.00632,18.0,2.3...|24.0|
|[0.01096,55.0,2.2...|22.0|
|[0.0136,75.0,4.0,...|18.9|
|[0.01381,80.0,0.4...|50.0|
|[0.01439,60.0,2.9...|29.1|
+--------------------+----+
only showing top 5 rows



In [22]:
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator

gbt = GBTRegressor(featuresCol="X", labelCol='Y', maxIter=10)
model = gbt.fit(trainingData)
predictions = model.transform(testData)

# Select example rows to display.
predictions.show(5)

# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="Y", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

# gbtModel = model.stages[1]
# print(gbtModel) 

+--------------------+----+------------------+
|                   X|   Y|        prediction|
+--------------------+----+------------------+
|[0.01381,80.0,0.4...|50.0| 50.28995603000262|
|[0.01432,100.0,1....|31.6|29.368586613172965|
|[0.01439,60.0,2.9...|29.1|  28.8793675356106|
|[0.01951,17.5,1.3...|33.0|33.625703453545526|
|[0.02729,0.0,7.07...|34.7|31.826898410579474|
+--------------------+----+------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 4.15822
